In [1]:
import os
import json
import pandas as pd
import traceback


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain.chat_models import ChatOpenAI

In [7]:
key = os.getenv('OPEN_API_KEY')

In [11]:
llm= ChatOpenAI(openai_api_key=key, model_name='gpt-3.5-turbo',temperature=0.5)

In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PipelinePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [14]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [15]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker, Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your respose like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [18]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [19]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [20]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [21]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [22]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review", verbose=True)

In [23]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [24]:
file_path="/Users/aliyuyusuf/mcqgen/data.txt"

In [25]:
with open(file_path,'r') as file:
    TEXT = file.read()
    

In [27]:
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [28]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [29]:
with get_openai_callback() as cb:
    response= generate_evaluate_chain(
        {
            "text": TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:What is machine learning and how does it work? In-depth guide
By
Linda Tucci, Industry Editor -- CIO/IT Strategy Ed Burns
Machine learning (ML) is a type of artificial intelligence (AI) focused on building computer systems that learn from data. The broad range of techniques ML encompasses enables software applications to improve their performance over time.

Machine learning algorithms are trained to find relationships and patterns in data. They use historical data as input to make predictions, classify information, cluster data points, reduce dimensionality and even help generate new content, as demonstrated by new ML-fueled applications such as ChatGPT, Dall-E 2 and GitHub Copilot.
Machine learning is widely applicable across many industries. Recommendation engines, for example, are used by e-commerce, social media and news organizations to suggest content based on a customer's

In [30]:
print(f'Total tokens:{cb.total_tokens}')
print(f'Prompt Tokens:{cb.prompt_tokens}')
print(f'Completion Tokens:{cb.completion_tokens}')
print(f'Total Cost:{cb.total_cost}')

Total tokens:7619
Prompt Tokens:6946
Completion Tokens:673
Total Cost:0.011765


In [31]:
response

{'text': 'What is machine learning and how does it work? In-depth guide\nBy\nLinda Tucci, Industry Editor -- CIO/IT Strategy Ed Burns\nMachine learning (ML) is a type of artificial intelligence (AI) focused on building computer systems that learn from data. The broad range of techniques ML encompasses enables software applications to improve their performance over time.\n\nMachine learning algorithms are trained to find relationships and patterns in data. They use historical data as input to make predictions, classify information, cluster data points, reduce dimensionality and even help generate new content, as demonstrated by new ML-fueled applications such as ChatGPT, Dall-E 2 and GitHub Copilot.\nMachine learning is widely applicable across many industries. Recommendation engines, for example, are used by e-commerce, social media and news organizations to suggest content based on a customer\'s past behavior. Machine learning algorithms and machine vision are a critical component of 

In [33]:
quiz =response.get('quiz')

In [34]:
json.loads(quiz)

{'1': {'mcq': 'What is the primary focus of machine learning?',
  'options': {'a': 'Building computer systems that learn from data',
   'b': 'Creating artificial intelligence robots',
   'c': 'Developing software applications for gaming',
   'd': 'Designing self-driving cars'},
  'correct': 'a'},
 '2': {'mcq': 'Which type of machine learning algorithm does not require labeled data?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Semisupervised learning',
   'd': 'Reinforcement learning'},
  'correct': 'b'},
 '3': {'mcq': 'What is one of the common use cases of machine learning in the healthcare industry?',
  'options': {'a': 'Fraud detection',
   'b': 'Spam filtering',
   'c': 'Diagnosing and suggesting treatment plans',
   'd': 'Predictive maintenance'},
  'correct': 'c'},
 '4': {'mcq': 'Which industry uses machine learning for personalization, inventory management, and demand forecasting?',
  'options': {'a': 'Financial services',
   'b': 'Pharma